EJERCICIO 1

In [ ]:
from robobopy.Robobo import Robobo
from robobopy.utils.Wheels import Wheels
from robobopy.utils.BlobColor import BlobColor
IP_ROBOBO="192.168.1.140"
rob = Robobo(IP_ROBOBO)
rob.connect()

#Usandolo de esta manera solo se habilitará color "CUSTOM"
#Recordatorio de que hay que calibrar antes la cámara
rob.setActiveBlobs(False, False, False, True)


rob.moveTiltTo(85,15)

tiempo=0
while tiempo<240:

    print(rob.readColorBlob(BlobColor.CUSTOM).size)
    print(rob.readColorBlob(BlobColor.CUSTOM).posx)
    if rob.readColorBlob(BlobColor.CUSTOM).size>5:
        if rob.readColorBlob(BlobColor.CUSTOM).posx<30:
            rob.moveWheels(5,-5)
        elif rob.readColorBlob(BlobColor.CUSTOM).posx>70:
            
            rob.moveWheels(-5,5)
        else:
            rob.stopMotors()    




    rob.wait(0.2)
    tiempo+=1

rob.stopMotors()
rob.disconnect()


EJERCICIO 2

In [ ]:
from robobopy.Robobo import Robobo
from robobopy.utils.Emotions import Emotions
IP_ROBOBO="192.168.1.140"
rob = Robobo(IP_ROBOBO)
rob.connect()

#Retraso para darle margen y poder leer la batería sin problema
rob.wait(1)
#Robobo dice el nivel de batería de la base y el del teléfono

tiempo=0
while tiempo<60:
    rob.sayText("El nivel de batería del teléfono es " + str(rob.readBatteryLevel('phone')))
    V_giro=3 + (rob.readBatteryLevel('base'))/5

    if rob.readBatteryLevel('phone')>20:
        rob.moveWheels(V_giro,-V_giro)  
        rob.setEmotionTo(Emotions.SLEEPING)
    elif rob.readBatteryLevel('phone')>20 and rob.readBatteryLevel('phone')>=10:
        rob.moveWheels(V_giro,-V_giro) 
        rob.setEmotionTo(Emotions.TIRED)
    elif rob.readBatteryLevel('phone')<10:
        rob.setEmotionTo(Emotions.SLEEPING)
        rob.stopMotors()

    rob.wait(1)
    tiempo+=1

rob.stopMotors()
rob.disconnect()


EJERCICIO 3

In [ ]:
from robobopy.Robobo import Robobo
from robobopy.utils.Sounds import Sounds
from robobopy.utils.Emotions import Emotions

#Conexión con el Robobo real
IP_ROBOBO="192.168.1.140"
rob = Robobo(IP_ROBOBO)
rob.connect()

#Variable del tiempo de ejecución
timeout=0
rob.setEmotionTo(Emotions.SLEEPING)
OFF_ON=False

while timeout<300:
    print(f"{rob.readClapCounter()} palmadas")
    #Si dió más de 1 palmada, Robobo reaccionará
    if rob.readClapCounter() >= 3:
        if not OFF_ON:
            rob.moveWheels(10,10)
            rob.setEmotionTo(Emotions.HAPPY)
            rob.wait(0.25)
            OFF_ON=True
        else:
            rob.stopMotors()
            rob.setEmotionTo(Emotions.SLEEPING)
            rob.wait(0.25)
            OFF_ON=False
        rob.resetClapCounter()
    rob.wait(0.1)
    timeout+=1

#Se desconecta y termina el programa
rob.stopMotors()
rob.disconnect()

EJERCICIO 4

In [ ]:
from robobopy.Robobo import Robobo
from robobopy.utils.Emotions import Emotions
IP_ROBOBO="192.168.1.140"
rob = Robobo(IP_ROBOBO)
rob.connect()

#Retraso para darle margen y poder leer la batería sin problema
rob.wait(1)
#Robobo dice el nivel de batería de la base y el del teléfono

tiempo=0
rob.moveTiltTo(95,15)
rob.moveWheels(-5,5)
Move=False
while tiempo<240:


    print(rob.readBrightnessSensor())

    if rob.readBrightnessSensor()>20:
        rob.moveWheels(10,10)
        Move=True


    if Move and rob.readBrightnessSensor()>50:
        rob.stopMotors()
    elif Move and rob.readBrightnessSensor()<20:
        rob.stopMotors()
        rob.sayText("He perdido la luz. Voy a encontrar una nueva")
        Move=False
        rob.moveWheels(-5,5)

    rob.wait(0.25)
    tiempo+=1

rob.stopMotors()
rob.disconnect()

EJERCICIO 5

In [ ]:
from robobopy.Robobo import Robobo
from robobopy.utils.Emotions import Emotions
from robobopy.utils.IR import IR
#Función que se usará como callback. Detiene los motores completamente
def clap():
    #Variable global que hace que una palmada lo detenga, y la otra arranque
    global palmada
    if not palmada:
        rob.stopMotors()
        palmada=True
    else:
        rob.moveWheels(10,10)
        palmada=False

#Conexión a Robobo
IP_ROBOBO="192.168.1.140"
rob = Robobo(IP_ROBOBO)
rob.connect()


#Programa principal
def programa():
# El programa se mantiene en ejecución continua durante 30 segundos
    tiempo=0
    palmada=False


    #Robobo se mueve de forma constante
    rob.moveWheels(10,10)
    rob.wait(0.25)
    #Declaración de la función de interupción
    rob.whenClapIsDetected(clap)

    while tiempo<300:
        L=rob.readIRSensor(IR.FrontL)
        R=rob.readIRSensor(IR.FrontR)
        print(f"{L} {R}")
        if L<15 or R<15:
            rob.moveWheels(-20,15)
            while L<10 or R<10:
                L=rob.readIRSensor(IR.FrontL)
                R=rob.readIRSensor(IR.FrontR)
                rob.wait(0.1)
            if not palmada:
                rob.moveWheels(10,10)
    
        rob.wait(0.1)
        tiempo+=1

    #Detiene los motores en caso de que no se haya hecho antes
    rob.stopMotors()
    rob.sayText("Se acabó el trabajo de hoy")
    rob.setEmotionTo(Emotions.SLEEPING)
    #Desconexión del robot y finalización del programa
    rob.disconnect()
    
    

if __name__=="__main__":
    programa()


